In [1]:
import torch

# let's say these are token embedding vectors
inputs = torch.tensor (
    [[0.43, 0.15, 0.89],
     [0.55, 0.87, 0.66],
     [0.57, 0.85, 0.64],
     [0.22, 0.58, 0.33],
     [0.77, 0.25, 0.10],
     [0.05, 0.80, 0.55]]
)

In [2]:
# Then this is how you compute attention scores

query = inputs[1]

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(query, x_i)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [4]:
# Normalize the attention scores
attn_weights_2_tmp = attn_scores_2 / torch.sum(attn_scores_2)
print("Attention weights: ", attn_weights_2_tmp)
print("Sum of attention weights: ", torch.sum(attn_weights_2_tmp))

Attention weights:  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum of attention weights:  tensor(1.0000)


In [10]:
# Utilizing softmax function

def softmax_naive(x):
    return torch.exp(x) / torch.sum(torch.exp(x))

attn_weights_2_naive = softmax_naive(attn_scores_2)
print("Attention weights: ", attn_weights_2_naive)
print("Sum of attention weights: ", torch.sum(attn_weights_2_naive))

Attention weights:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum of attention weights:  tensor(1.)


In [6]:
# just using torch softmax directly

attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights: ", attn_weights_2)
print("Sum of attention weights: ", torch.sum(attn_weights_2))

Attention weights:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum of attention weights:  tensor(1.)


In [21]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i

print("Context vector: ", context_vec_2)

Context vector:  tensor([0.4419, 0.6515, 0.5683])
